<a href="https://colab.research.google.com/github/adnaneaabbar/stock-trading-system/blob/master/stock_trading_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nous allons tout d'abord faire l'étude, sur la banque d'investissement GS - Goldman Sachs Inc. Nous allons essayer de prédire à l'aide de séries chronologiques les prix de leurs actions.

---

# Etude à base de modèle LSTM


---

## Etape 1 : Chargement et analyse des données.

Nous utiliserons les données historiques des 10 dernières années du cours des actions de GS. Vous pouvez trouver les données sur yahoo finance.


In [0]:
import os
import sys
import time
import numpy as np
import pandas as pd 

In [0]:
url = 'https://raw.githubusercontent.com/adnaneaabbar/stock-trading-system/master/data/goldman_sachs.csv'
gs = pd.read_csv(url)

In [9]:
gs.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-12-31,167.289993,170.130005,166.929993,168.839996,146.078110,6401800
1,2010-01-04,170.050003,174.250000,169.509995,173.080002,149.746597,9135000
2,2010-01-05,173.000000,176.259995,172.570007,176.139999,152.394012,11659400
3,2010-01-06,175.380005,175.380005,173.759995,174.259995,150.767426,7381100
4,2010-01-07,174.320007,178.750000,173.949997,177.669998,153.717728,8727400


In [10]:
gs.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2260,2018-12-24,159.000000,160.000000,154.309998,156.350006,152.260864,3783500
2261,2018-12-26,157.000000,163.110001,151.699997,162.929993,158.668777,7054700
2262,2018-12-27,160.119995,165.410004,159.020004,165.410004,161.083923,4973000
2263,2018-12-28,165.639999,165.949997,162.020004,163.029999,158.766159,4110500
2264,2018-12-31,163.779999,167.119995,163.779999,167.050003,162.681030,4550000
